<span style="color:red">Abgegeben von (Name, Vorname):</span>

---


Elsherif, Mohamed

<p style="line-height:1.4">
<font size="6"><strong>3. Sitzung: Rechtschreibkorrektur & morphologische Ähnlichkeitsmaße</strong></font>
</p>

In diesem Notebook werden wir uns wieder mit Wortformen beschäftigen. Allerdings werden wir die Wortformen diesmal nicht vereinheitlichen, sondern deren Unterschiede bzw. Ähnlichkeiten messen.

Ähnlichkeitsmaße bei Wortformen spielen eine wichtige Rolle bei vielen Verfahren der NLP, insbesondere bei der **Rechtschreibkorrektur**, die wir in diesem Notebook ebenfalls behandeln werden.

# Rechtschreibkorrektur: Genereller Aufbau

Verfahren zur Rechtschreibkorrektur haben mindestens die folgenden Komponenten:

- Eingabe: Wortform (ggf. mit weiterem Kontext)
- Wissen: Korrekte Wortformen
- Ähnlichkeitsmaß
- Ausgabe: Korrekturvorschläge

<div>
<img src="attachment:spellchecking.png" width="500"/>
</div>

Im Prinzip funktionieren so auch aktuelle Korrekturhilfen wie [Aspell](http://aspell.net/man-html/Aspell-Suggestion-Strategy.html#Aspell-Suggestion-Strategy) oder [Hunspell](https://hunspell.github.io/). Wir werden im Folgenden sehen, wie das für das Englische umgesetzt werden kann.

# Ähnlichkeitsmaße bei Wortformen

Grundsätzlich gilt:

    Ähnlichkeitsmaß = Ähnlichkeitseigenschaft(en) + Gewichtung(en)
    
Eine **Ähnlichkeitseigenschaft** kann zum Beispiel sein:
- Identität (offensichtlich zu trivial)
- Länge des längsten gemeinsamen Teilstrings (Longest Common Substring)
- Mächtigkeit/Kardinalität der Schnittmenge der gemeinsamen Teilstrings (Jaccard Distance)
- Länge der Sequenz von Änderungsoperationen (Edit Distance)

Eine **Gewichtung** kann sich zum Beispiel ergeben aus:
- Länge der Wortformen
- Wahrscheinlichkeit der Wortformen
- Nähe der Buchstaben auf der Tastatur

## Longest Common Substring (LCS)

Der längster gemeinsame Teilstring (Longest Common Substring, LCS) lässt sich folgendermaßen definieren:

$$LCS(w_1,w_2) = \underset{w \trianglelefteq w1, w \trianglelefteq w2}{\operatorname{arg\,max}} |w|$$

Strenggenommen können zwei Strings mehrere LCS besitzen, aber das kümmert uns hier nicht, denn wir sind nur an der Länge eines LCS interessiert.

Zur Berechnung des LCS benötigen wir zunächst eine Funktion, die die Teilstrings eines Strings ausgibt:

In [1]:
def set_of_substrings(string, minlen=1, maxlen=999):
    return set([string[i: j]
                for i in range(len(string))
                for j in range(i + 1, len(string) + 1)
                if j-i >= minlen and j-i <= maxlen
                ])


print(set_of_substrings("execution"))

{'xec', 'xecuti', 'execution', 'xecu', 'io', 'ecu', 'c', 'execut', 'utio', 'xecution', 'executi', 'executio', 'cu', 't', 'uti', 'ution', 'ti', 'tion', 'tio', 'execu', 'i', 'ex', 'ut', 'x', 'exe', 'ion', 'ec', 'xecut', 'cuti', 'on', 'xe', 'n', 'ecutio', 'ecut', 'ecuti', 'xecutio', 'exec', 'u', 'o', 'cution', 'ecution', 'cut', 'e', 'cutio'}


Diese Funktion hat offensichtlich eine quadratische Laufzeit (und Speicherverbrauch) in der Länge der Eingabe – nicht schön, aber für unsere Zwecke reicht das.

Damit können wir die Menge der gemeinsamen Teilstrings und schließlich den längsten gemeinsamen Teilstring berechnen:

In [2]:
def common_set_of_substrings(string1, string2, minlen=1, maxlen=999):
    return set_of_substrings(string1, minlen, maxlen) & set_of_substrings(string2, minlen, maxlen)


print(common_set_of_substrings("execution", "intention"))

{'t', 'ti', 'tion', 'o', 'io', 'tio', 'n', 'on', 'i', 'e', 'ion'}


In [3]:
def longest_common_substring(string1, string2):
    css = common_set_of_substrings(string1, string2)
    if len(css) > 1:
        return max(css, key=len)
    else:
        return ""


print(longest_common_substring("execution", "intention"))

tion


<span style="color:red">Frage am Rande:</span> Wie ließe sich der längste gemeinsame Teilstring in linearer Zeit (in der Länge der Eingabe) ermitteln? Wäre das auf jeden Fall besser?

Was die Gewichtung betrifft, bietet sich die Länge des kleineren Strings an:

In [4]:
def lcs_sim(string1, string2):
    return len(longest_common_substring(string1, string2)) / min(len(string1), len(string2))


lcs_sim("execution", "intention")

0.4444444444444444

Das Problem ist hier natürlich, dass die Unterschiede zwischen zwei Wörtern so verteilt sein können, dass dieses Ähnlichkeitsmaß bei diesen Wörtern trotz der offensichtlichen Ähnlichkeit verhältnismäßig gering ausfällt.

Beispiele:

In [5]:
w1 = "execution"
w2 = "execUtion"
print("{}, {}: {}".format(w1,w2,lcs_sim(w1,w2)))

execution, execUtion: 0.4444444444444444


LCS ist also ein simples aber auch etwas "naives" Maß für die Ähnlichkeit zweier Wörter.

## Teilstring-Überlappung (aka Jaccard-Distanz)

Ein anderes Ähnlichkeitsmaß, bekannt als **Jaccard-Distanz (JD)**, betrachtet die *Überschneidung* der Mengen der Teilstrings und gewichtet diese anhand der *Vereinigung* der beiden Mengen:

$$jd(A,B) = \frac{|A \cap B|}{|A \cup B|}$$

Hier wären $A$ und $B$ die Mengen der Substrings zweier Wortformen.

Mit den oben eingeführten Funktionen lässt sich dies direkt umsetzen:

In [6]:
def jd_sim(string1, string2):
    return len(common_set_of_substrings(string1, string2)) / len(set_of_substrings(string1) | set_of_substrings(string2))


w1 = "execution"
w2 = "intention"
print("{}, {}: {}".format(w1, w2, jd_sim(w1,w2)))

w1 = "execution"
w2 = "execUtion"
print("{}, {}: {}".format(w1, w2, jd_sim(w1,w2)))

execution, intention: 0.1506849315068493
execution, execUtion: 0.2753623188405797


Die JD-Ähnlichkeit zwischen *execution* und *execUtion* ist nun erfreulicherweise größer als die zwischen *execution* und *intention* – im Unterschied zur LCS-Ähnlichkeit.

## <span style="color:red">Aufgaben I</span>

<span style="color:red">A1:</span> Implementieren Sie eine toleranteres Ähnlichkeitsmaß auf Grundlage von LCS als `almost_lcs_sim()`, indem Sie im längsten gemeinsamen Teilstring kleinere Abweichungen zulassen, zum Beispiel hinsichtlich:

- Groß-Kleinschreibung
- benachbarte Buchstaben auf der Tastatur
- Verdopplungen
- Auslassungen
- Affigierung
- ...

Diese müssen natürlich anders gewichtet werden als die echten Teilstrings!

In [27]:
def almost_lcs_sim(string1, string2):
    out = "Ätsch!"
    # Dictionary für benachbarte Tasten auf einer typischen QWERTZ-Tastatur mit ChatGPT generiert
    # Um kleinere Tippfehler zu identifizieren, bei denen möglicherweise Tasten in der Nähe mistakenly gedrückt wurden
    Tasta_Nachbarn = {
        'q': 'wa', 'w': 'qes', 'e': 'wsd', 'r': 'edf', 't': 'rfgy', 'z': 'tyu', 'u': 'yzih', 'i': 'ujko', 'o': 'iklp', 'p': 'ol',
        'ü': 'io', 'a': 'qwsz', 's': 'awedxz', 'd': 'serfcx', 'f': 'drtgv', 'g': 'ftyhbv', 'h': 'gyujbn', 'j': 'huikmn', 'k': 'jiolm', 'l': 'kop',
        'ö': 'lp', 'ä': 'öp', 'y': 'zu', 'x': 'zsdc', 'c': 'xdfv', 'v': 'cfgb', 'b': 'vghn', 'n': 'bhjm', 'm': 'njk',
        'ß': 'm',
        '1': '2q', '2': '13w', '3': '24e', '4': '35r', '5': '46t', '6': '57z', '7': '68u', '8': '79i', '9': '8o', '0': '9p',
        '-': '=p', '=': '-+',
        '!': '1q', '@': '2w', '#': '3e', '$': '4r', '%': '5t', '&': '6z', '/': '7u', '(': '8i', ')': '9o', '=': '0p'
    }
    # Die Großbuchstaben der Tastaturen auch Berücksichtigen
    for k in list(Tasta_Nachbarn.keys()):
        Tasta_Nachbarn[k.upper()] = Tasta_Nachbarn[k].upper()

    # Function zur Feststellung, ob zwei charchters ähnlich sind
    # Characters sind ähnlich wenn sie identical sind (case-insensitive)
    # oder Nachbarn auf der Tastatur sind (according to `Tasta_Nachbarn`)
    def sim_chk(char1, char2):
        # Überprüfen, ob charachters genau übereinstimmen, wenn die Groß-/Kleinschreibung ignoriert wird
        if char1.lower() == char2.lower():
            return True
        # Überprüfen, ob die zweite charachter char2 ein Nachbar von char1 auf der Tastatur ist
        if char2 in Tasta_Nachbarn.get(char1, ""):
            return True
        return False

    # Function um longest common substring zu berechnen
    # die bestimme fehler erlauben
    def modified_longest_common_substring(string1, string2):
        # beide strings in Kleinbuchstaben umwandeln
        string1, string2 = string1.lower(), string2.lower()

        # Länge den beiden strings ermittlen um DP tabelle dimentionen zu finden
        m, n = len(string1), len(string2)

        # 2D-DP-Tabelle initializieren
        # Tisch[i][j] will hold the longest common substring length ending at string1[i-1] and string2[j-1]
        Tisch = [[0] * (n + 1) for _ in range(m + 1)]

        # zum Speichern der langeste gefundenen string
        max_len = 0

        # Die DP Tabelle füllen
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                # Aktuelle charchters in string1 und string2
                char1, char2 = string1[i - 1], string2[j - 1]

                # Wenn charachters genau übereinstimmen, verlängern die substring lenght mit +1 gewischt
                if char1 == char2:
                    Tisch[i][j] = Tisch[i - 1][j - 1] + 1
                    max_len = max(max_len, Tisch[i][j])        # Update max_len if new max is found

                # wenn characters sind ähnlich aufgrund der Tastaturnähe oder der Groß-/Kleinschreibung
                # erweitern the substring length mit 0.8 gewischt
                elif sim_chk(char1, char2):
                    Tisch[i][j] = Tisch[i - 1][j - 1] + 0.8

                # Behandeln Duplikate, die auf einen Tippfehler zurückzuführen sind
                # indem ein wiederholtes charhcter in string1 erkennen das in string2 nicht dupliziert ist, deshalb - 0.2 gewischt
                elif i > 1 and string1[i - 1] == string1[i - 2] and (j == 1 or string2[j - 1] != string2[j - 2]):
                    Tisch[i][j] = max(Tisch[i][j], Tisch[i - 1][j] - 0.2)

                # Behandeln Auslassungen indem die vorherige beste match erweitern
                elif i > 1 and Tisch[i - 1][j] > 0:
                    Tisch[i][j] = Tisch[i - 1][j] - 0.5
                elif j > 1 and Tisch[i][j - 1] > 0:
                    Tisch[i][j] = Tisch[i][j - 1] - 0.5

                # Behandeln Duplikate, bei denen beide strings dasselbe consecutive characters enthalten wie zB das wort
                #'Book' oder 'Cheese' (bzw das ist kein Tippfehler), deshalb sie korrekt/ähnlich sind mit + 1 gewischt
                if i > 1 and string1[i - 1] == string1[i - 2] and j > 1 and string2[j - 1] == string2[j - 2]:
                    Tisch[i][j] = max(Tisch[i][j], Tisch[i - 2][j - 2] + 1)

        # maximale Länge des gefundenen substring zuruckgeben
        return max_len

    # Die LCS-Länge berechnen
    lcs_len = modified_longest_common_substring(string1, string2)

    # Normalizierung der LCS Länge durch die kurzen string um similarity score zu berechnen
    similarity_score = lcs_len / min(len(string1), len(string2))
    out = similarity_score

    return out

In [28]:
# Tests für A1 (Bitte nicht ändern!)

testset = [("execution", "execution"),
  ("execution", "intention"),
  ("inetntion", "intention"),
  ("execUtion", "execution"),
  ("exxecution", "execution"),
  ("eecution", "execution"),
  ("execuzion", "execution"),
  ("Hunden", "Hündin")]

[(i, j, almost_lcs_sim(i, j)) for i, j in testset]

[('execution', 'execution', 1.0),
 ('execution', 'intention', 0.4444444444444444),
 ('inetntion', 'intention', 0.7777777777777778),
 ('execUtion', 'execution', 1.0),
 ('exxecution', 'execution', 0.9444444444444444),
 ('eecution', 'execution', 0.9375),
 ('execuzion', 'execution', 0.9777777777777779),
 ('Hunden', 'Hündin', 0.35000000000000003)]

Zum Vergleich:

In [11]:
[(i, j, lcs_sim(i, j)) for i, j in testset]

[('execution', 'execution', 1.0),
 ('execution', 'intention', 0.4444444444444444),
 ('inetntion', 'intention', 0.5555555555555556),
 ('execUtion', 'execution', 0.4444444444444444),
 ('exxecution', 'execution', 0.8888888888888888),
 ('eecution', 'execution', 0.875),
 ('execuzion', 'execution', 0.5555555555555556),
 ('Hunden', 'Hündin', 0.3333333333333333)]

## Edit Distance (aka Levenshtein Distance)

Eine ganz andere Ähnlichkeitseigenschaft von Wortformen ist die **Edit Distance** (Editierdistanz, [Levenshtein Distance](https://de.wikipedia.org/wiki/Levenshtein-Distanz)).

### Funktionsweise

Die Idee ist, dass es eine kleine Menge von Operationen gibt, nämlich `{insert, delete, substitute, keep}`, die auf Buchstaben angewandt werden können, um ein Wort A in ein Wort B umzuwandeln.

Das folgende Beispiel (aus Jurafsky & Martin 2021) stellt eine solche Konversion von *intention* nach *execution* dar:

| i | n   | t   | e   | *   | n   | t   | i   | o   | n   |
|---|-----|-----|-----|-----|-----|-----|-----|-----|-----|
| ↓ | ↓   | ↓   | ↓   | ↓   | ↓   | ↓   | ↓   | ↓   | ↓   |
| * | **e** | **x** | **e** | **c** | **u** | **t** | **i** | **o** | **n** |
| D | S   | S   |     | I   | S   |     |     |     |     |      |              |

Die Konversionsoperationen können dabei unterschiedlich gewichtet werden:
- D(elete): 1
- S(substitute) : 2
- I(nsert): 1
- (keep): 0

Die Operation S ist deshalb "teurer" als D und I, weil hier quasi implizit D und I durchgeführt werden.

Aus den angewandten Operationen und deren Gewicht ergibt sich dann, dass die Edit Distance zwischen *intention* und *execution* 8 beträgt.

### Minimierung der Edit Distance

(im Wesentlichen übernommen aus Jurafsky & Martin 2021)

Nun gibt es für zwei Wortformen immer trivialerweise unendlich viele mögliche Werte als Edit Distance – wir können ja beliebig Buchstaben hinzufügen und löschen. Aus diesen möglichen Distanzen interessiert uns aber eigentlich nur die *minimale* Distanz, die sogenannte Minimal Edit Distance oder $D_{Lev}$.

Zur Berechnung von $D_{Lev}$ greift man auf die Methode der **Dynamischen Programmierung** zurück. D.h. man merkt sich Zwischenergebnisse und berechnet nicht für jeden Edit Path alles neu.

Dafür wird eine Matrix/Tabelle erstellt, so dass die Buchstaben des einen Worts die Zeilen definieren und die Buchstaben des anderen Worts die Spalten. (Es spielt keine Rolle, welche Wortform in den Spalten und welche Wortform in den Zeilen steht.)

Nehmen wir im Folgenden an, dass $w_1 =$ *execution* und $w_2 =$ *intention*. Das jeweils erste Zeichen ist das leere Zeichen \# und die Tabelle wird in der Zelle $[1,1]$ mit $0$ initialisiert.  

|       | \# | e | x | e | c | u | t | i | o | n |
|-------|----|---|---|---|---|---|---|---|---|---|
| \#    |  0 |   |   |   |   |   |   |   |   |   |
| **i** |    |   |   |   |   |   |   |   |   |   |
| **n** |    |   |   |   |   |   |   |   |   |   |
| **t** |    |   |   |   |   |   |   |   |   |   |
| **e** |    |   |   |   |   |   |   |   |   |   |
| **n** |    |   |   |   |   |   |   |   |   |   |
| **t** |    |   |   |   |   |   |   |   |   |   |
| **i** |    |   |   |   |   |   |   |   |   |   |
| **o** |    |   |   |   |   |   |   |   |   |   |
| **n** |    |   |   |   |   |   |   |   |   |   |

Im Anschluss wird die Tabelle von $[1,1]$ bis $[|w_1|+1,|w_2|+1]$ anhand der rekursiven Funktion $D$ ausgefüllt:

$$\begin{array}{ll}
D(x,y) = min ( & D(x-1,y) + \text{del-cost},\\
& D(x,y-1) + \text{ins-cost},\\
& D(x-1,y-1) + \text{subst-cost},\\
& D(x-1,y-1) ~\mathit{iff}~ w_1[x]=w_2[y])
\end{array}$$

Dabei soll gelten:

$$\begin{array}{ll}
\text{del-cost} = 1\\
\text{ins-cost} = 1\\
\text{subst-cost} = 2
\end{array}$$

Wir erhalten schließlich die folgende vollständig ausgefüllte Tabelle:

|       | \#  | e   | x   | e    | c    | u    | t    | i    | o    | n    |
|-------|-----|-----|-----|------|------|------|------|------|------|------|
| \#    | <span style="color:red">0</span> | 1 | 2 | 3  | 4  | 5  | 6  | 7  | 8  | 9  |
| **i** | <span style="color:red">1</span> | 2 | 3 | 4  | 5  | 6  | 7  | 6  | 7  | 8  |
| **n** | 2 | <span style="color:red">3</span> | 4 | 5  | 6  | 7  | 8  | 7  | 8  | 7  |
| **t** | 3 | 4 | <span style="color:red">5</span> | 6  | 7  | 8  | 7  | 8  | 9  | 8  |
| **e** | 4 | 3 | 4 | <span style="color:red">5</span>  | <span style="color:red">6</span>  | 7  | 8  | 9  | 10 | 9  |
| **n** | 5 | 4 | 5 | 6  | 7  | <span style="color:red">8</span>  | 9  | 10 | 11 | 10 |
| **t** | 6 | 5 | 6 | 7  | 8  | 9  | <span style="color:red">8</span>  | 9  | 10 | 11 |
| **i** | 7 | 6 | 7 | 8  | 9  | 10 | 9  | <span style="color:red">8</span>  | 9  | 10 |
| **o** | 8 | 7 | 8 | 9  | 10 | 11 | 10 | 9  | <span style="color:red">8</span>  | 9  |
| **n** | 9 | 8 | 9 | 10 | 11 | 12 | 11 | 10 | 9  | <span style="color:red">8</span>  |

$D_{Lev}(w_1,w_2)$ ist dann der Inhalt der Zelle $[|w_1|+1,|w_2|+1]$, also $8$. Die rot hervorgehobenen Zellen stellen einen Minimal-Edit-Pfad dar, den man anhand der Backtraces rekonstruieren kann.

### Edit Distance in NLTK

NLTK stellt die entsprechende Funktion `edit_distance(s1,s2)` im Modul [`metrics`](https://www.nltk.org/api/nltk.metrics.html#module-nltk.metrics.distance) zur Verfügung.

Die Gewichtung der Substitutionsoperation kann mittels des Parameters `substitution_cost` spezifiziert werden:

In [12]:
from nltk.metrics import edit_distance

[(i, j, edit_distance(i, j, substitution_cost=2)) for i, j in testset]

[('execution', 'execution', 0),
 ('execution', 'intention', 8),
 ('inetntion', 'intention', 2),
 ('execUtion', 'execution', 2),
 ('exxecution', 'execution', 1),
 ('eecution', 'execution', 1),
 ('execuzion', 'execution', 2),
 ('Hunden', 'Hündin', 4)]

Wieder kann die Ähnlichkeitseigenschaft mittels der Länge der kleineren Wortform gewichtet werden:

In [13]:
def lev_sim(string1, string2):
    return 1 - edit_distance(string1, string2) / len(min(string1, string2))


[(i, j, lev_sim(i, j)) for i, j in testset]

[('execution', 'execution', 1.0),
 ('execution', 'intention', 0.4444444444444444),
 ('inetntion', 'intention', 0.7777777777777778),
 ('execUtion', 'execution', 0.8888888888888888),
 ('exxecution', 'execution', 0.8888888888888888),
 ('eecution', 'execution', 0.875),
 ('execuzion', 'execution', 0.8888888888888888),
 ('Hunden', 'Hündin', 0.6666666666666667)]

# Rechtschreibkorrektur mit Edit Distance und dem Brown Corpus

Mit Hilfe der Edit Distance ist es recht einfach, ein Programm zu schreiben, das Vorschläge zur Rechtschreibkorrektur machen kann. Man braucht dafür im Grunde nur noch eine Liste mit Wortformen, die als Korrektur vorgeschlagen werden sollen.

Die Wortformenliste können wir einfach aus den Worttoken im Brown Corpus generieren. Außerdem nutzen wir eine Funktion `propose_word_corrections()`, die für einen Eingabestring diejenigen Wortformen aus einer Wortformenliste ausgibt, die maximal eine bestimmte Edit Distance (`edt`) zum Eingabestring haben. Als Funktion zur Ermittlung der Edit Distance dient uns diesmal `edit_distance()` aus NLTK.

In [14]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [15]:
from nltk.corpus import brown
from nltk.metrics import edit_distance

brownWords = set(brown.words())


def propose_word_corrections(string, edt, lexicon):
    return [word for word in lexicon if edit_distance(word, string) <= edt]

Mit `propose_word_corrections()` erhalten wir relativ zügig sinnvolle Vorschläge zur Rechtschreibkorrektur.

In [16]:
propose_word_corrections("inetntion", 2, brownWords)

['invention', 'intention']

Was nun noch stört, ist die Edit Distance Threshold `edt`. Wie könnte die automatisch gesetzt werden?

## <span style="color:red">Aufgaben II</span>

<span style="color:red">A2:</span> Implementieren Sie eine Funktion `lazy_propose_word_corrections(string,lexicon)`, die `edt` automatisch (und möglichst sinnvoll) anhand der Eingabe und der Ausgabe festlegt. Führen Sie damit den anschließenden Test aus.

In [17]:
# Lösung A2

def lazy_propose_word_corrections (string,lexicon) :
  # „edt“ auf der Grundlage der Länge der input string definieren
    if len(string) <= 3:
        edt = 1
    elif len(string) <= 6:       # Diese Werte habe ich nach mehreren Versuch/Irrtum-Experimenten gewählt
        edt = 1
    elif len(string) <= 10:
        edt = 2
    else:
        edt = 3    # Für längere strings

    # Wörter aus dem Lexikon zurückgeben, die innerhalb der berechneten Edit distance liegen
    return [word for word in lexicon if edit_distance(word, string) <= edt]
    #return []

Test von `lazy_propose_word_corrections()`:

In [18]:
lazy_propose_word_corrections("inetntion",brownWords)

['invention', 'intention']

# Probleme und Verbesserungsmöglichkeiten

## Umfang der Wortformenliste

Die Korrekturvorschläge werden auf Grundlage einer Wortformenliste gemacht. Diese Wortformenliste sollte im Idealfall alle korrekt geschriebenen Worte einer Sprache enthalten. Oben haben wir dafür das Brown Corpus verwendet und für das Englische war das bereits eine gute Approximation.

Für eine Sprache wie das Deutsche gestaltet sich das aber schwieriger wegen der reichaltigeren Flexion und der Möglichkeit, Komposita ohne Leerzeichen zu bilden (*Eiersalatsoßengewürz*). Dadurch ist die Anzahl der Wortformen im Deutschen *wesentlich* höher als im Englischen.

Heutzutage scheint das kein Problem mehr zu sein, denn man kann einfach ein riesigs Webcorpus nehmen (z.B. enthält [DeReKo](https://www.ids-mannheim.de/digspra/kl/projekte/korpora/) mehr als 50 Milliarden Token), in dem Millionen von Wortformen enthalten sind. Das Problem ist aber dann, dass solche Ressourcen auch sehr untypische Wortformen enthalten, was die Korrekturhilfe einerseits zu tolerant macht und andererseits zu sehr vielen nutzlosen Vorschlägen führen kann. Und bestimmte Komposita werden auch dort nicht enthalten sein.

Ein Lösungsansatz besteht darin, mehrstufig vorzugehen: Die Komposita werden zuerst getrennt und dann überprüft.

Außerdem kann die Häufigkeit einer Wortform im Corpus verwendet werden, um die Liste der Korrekturvorschläge klein und möglichst sinnvoll zu halten.

## Geschwindigkeit

Der Umfang der Wortliste hat auch einen Einfluss auf die Geschwindigkeit der Rechtschreibkorrektur. Wenn so wie bei der  Funktion `propose_word_corrections()` die Edit Distance zu allen Wortformen der Wortliste einzeln berechnet wird, wird sich das auf die Geschwindigkeit gerade bei umfangreiche Wortlisten spürbar auswirken. Die Wortliste muss also irgendwie stärker strukturiert werden, um den Suchbereich sinnvoll einzugrenzen.

Zudem könnte man die Berechnung der Edit Distance früher abbrechen, wenn ein Schwellenwert bekannt ist und aus dem Durchlauf der Tabelle klar wird, dass dieser nicht mehr erreicht oder unterboten werden kann.

## Kontextabhängigkeit

Wortformen wie *dass* und *das* sind nicht an allen Positionen im Satz gleich gut. Ein "intelligenter" Rechtschreibkorrektor würde erkennen, dass *das* an der Stelle einer Konjunktion (*dass*, *weil*, *nachdem*, ...) falsch geschrieben ist. Dafür brauchen wir Informationen zum POS-Tag oder Informationen zum Kontext in einem Satz.

## Editieroperationen und deren Gewichte

Um die Qualität der Rechtschreibkorrektur zu erhöhen, können weitere Editieroperationen hinzugefügt und deren Gewichte optimiert werden.

Wir haben z.B. oben schon gesehen, dass Subsitution mal mit 1 und mal mit 2 gewichtet wird. Außerdem könnte man sich überlegen, die Gewichtung abhängig von der relativen Position auf der Tastatur festzulegen: Wenn wir eine Substitution von zwei Buchstaben wie *i* und *o* haben, dann sollte das geringer gewichtet werden als eine Substitution von *i* und *s*. Leider ist es gar nicht so trivial, die Gewichte relativ zueinander optimal festzulegen. Man wird hier wohl sehr viele Testdurchläufe brauchen (siehe unten) ...

Was die Editieroperationen betrifft, stellt `edit_distance()` zusätzlich noch die **Transposition** zur Verfügung. Diese Operationen fasst im Grunde eine Deletion und eine Insertion zusammen und ist damit der Substitution sehr ähnlich:

- *in**et**ntion* $\Rightarrow$ *in**te**ntion*

Bei `edit_distance()` kann die Transposition mit dem Argument `transpositions=True` aktiviert werden und wird dann mit 1 gewichtet.

In [19]:
print("without transpositions: " + str(edit_distance("inetntion","intention")))
print("with transpositions: "    + str(edit_distance("inetntion","intention",transpositions=True)))

without transpositions: 2
with transpositions: 1


Es sind natürlich noch weitere Editieroperationen denkbar. Z.B. wäre es manchmal hilfreich, ein Leerzeichen einfügen zu können.

- *allright* $\Rightarrow$ *all right*

Ein nützliches Kriterium kann außerdem die [graphophonologische Ähnlichkeit](http://aspell.net/metaphone/) sein:

- *taff* $\Rightarrow$ *tough*

# Evaluation durch Testdaten

Wir haben bis hierhin gesehen, dass es eine Reihe von Parametern (Ähnlichkeitseigenschaften, Editieroperationen, Gewichte, Wortliste, ...) gibt, die man bei der Implementierung einer Rechtschreibkorrektur festlegen muss. Die Frage ist, wie man das so machen kann, dass die Zielgrößen optimiert werden, nämlich:
- niedriger Ressourcenverbrauch: Zeit, Strom, Speicher
- hohe [Präzision](https://en.wikipedia.org/wiki/Precision_and_recall): $P = \frac{tp}{tp+fp}$
- hoher [Recall](https://en.wikipedia.org/wiki/Precision_and_recall): $R = \frac{tp}{tp+fn}$
- hohes [F1-Maß](https://en.wikipedia.org/wiki/F1_score): $F = \frac{2*P*R}{P+R}$

Außerdem müssen wir die Zielgrößen anhand konkreter Daten, dem sogenannten **Testset**, überprüfen. Es reicht nicht zu sagen: "Ich glaube aber, dass diese Parameter besser sind."

Tatsächlich wirft jedoch der Bedarf eines Testsets eine weitere nichttriviale Frage auf: Wie können wir ein Testset so zusammenstellen, dass die Qualität der Rechtschreibkorrektur *realistischerweise* gemessen werden kann. Tatsächlich erfordert das ein nicht unerhebliches Wissen hinsichtlich der Morphophonologie, der Tippergonomie, der typischen Fehler und der Erwartungshaltung der Nutzer ...

Wir werden im Folgenden die Qualität unserer Rechtschreibkorrektoren anhand eines Testsets überprüfen. Als Testset verwenden wir das Testset von Aspell .

## <span style="color:red">Aufgaben III</span>

<span style="color:red">A3:</span> Laden Sie das Testset von Aspell (http://aspell.net/test/cur/batch0.tab) herunter und verändern Sie die folgende Funktion `TEST_propose_word_corrections()` so, dass für eine Eingabe `s1` eine möglichst gute Korrekturliste hinsichtlich des F1-Maßes ausgegeben wird. Testen Sie anschließend die generierten Korrekturlisten mit dem darunter stehenden Skript.

In [ ]:
#### Der Vorgang mit dem Code dauert ca 57 Minuten ####     Recall: 0.6197; Precision: 0.124: F1-score: 0.2073

# Ich habe viele Strategien ausprobiert, die keine maschinelle Klassifikation oder vorab trainierte Sprachmodelle verwenden (da dies anscheinend nicht der Zweck dieser Aufgabe ist),
# um die Parameter zu verbessern/abzustimmen, die den besten F1-Score ergeben. Zu diesen Strategien gehörte das Filtern von Wörtern im Brown Corpus basierend auf dynamisch bestimmten Worthäufigkeiten
# und anschließend die Bewertung auf Grundlage einer Kombination aus dynamisch angepasster gewichteter Editierdistanz und gewichteter Häufigkeit.
# Dies sollte sicherstellen, dass die endgültige Liste die besten 5 oder 10 Korrekturen basierend auf Editierdistanz und Häufigkeit enthält.
# Ich hoffte, dass dies den F1-Score erhöhen würde. Es gelang mir, die Recall- und Precision-Werte zu verbessern, aber ich erreichte nie einen F1-Score von mehr als 0.2!
# Ein signifikant limitierender Faktor in dieser Aufgabe war die Rechenzeit. Die kürzeste war 18 Minuten und die längste 57 Minuten für die verschiedenen Ansätze,
# die ich versucht habe, um einen guten F1-Score zu erreichen.

from collections import Counter

def TEST_propose_word_corrections(s1):
    out = []

    # Worthäufigkeiten im Brown Corpus zahlen
    word_freq = Counter(brownWords)  # Ensure brownWords is a list of words from the Brown Corpus

    # edit distance basierend auf der Wortlänge anpassen
    if len(s1) <= 3:
        edt = [1, 2, 3]
    elif len(s1) <= 6:
        edt = [1, 2, 3, 4]
    elif len(s1) <= 10:
        edt = [2, 3, 4, 5]
    else:
        edt = [3, 4, 5, 6]

    # Set Initialisieren, der alle vorgeschlagenen Korrekturen enthält Initialisieren
    all_proposed_corrections = set()

    # Initialize frequency threshold for the Brown Corpus
    min_freq = 2  # Start with a low frequency threshold to allow rare words

    # über alle möglichen edit distance Iterieren
    for edit in edt:
        # Filtern von Wörtern aus Brown Corpus mit ausreichender Häufigkeit
        filtered_words = [w for w in brownWords if word_freq[w] >= min_freq]

        # Kandidatenkorrekturen mit der vorhandenen Funktion "propose_word_corrections" Generieren
        proposedCorrections = propose_word_corrections(s1, edit, filtered_words)
        all_proposed_corrections.update(proposedCorrections)

        # frequency threshold dynamisches Anpassen, wenn keine Kandidaten gefunden werden
        if not proposedCorrections and min_freq > 1:
            min_freq -= 1  # Gradually lower the threshold if no matches are found

        # Vorzeitiges Stoppen, wenn genügend Korrekturen gefunden wurden (Um unnecessary computation zu vermeiden)
        if len(all_proposed_corrections) >= 5:
            break

    # Dynamisches Anpassen der Gewichtung basierend auf length of the input word
    edit_weight = 1           # Grundgewicht for edit distance
    freq_weight = 0.1         # Grundgewicht for frequency in the Brown Corpus

    # die Gewichtung für den Edit distance dynamisch basierend auf der Wortlänge anpassen
    if len(s1) <= 3:
        edit_weight = 0.8   # Kurze Wörter haben eine geringere Gewichtung der Edit distance
    elif len(s1) <= 6:
        edit_weight = 1     # Medium-length words get a normal edit distance weight
    elif len(s1) <= 10:
        edit_weight = 1.2   # Längere Wörter benötigen möglicherweise etwas mehr Gewicht für den edit distance
    else:
        edit_weight = 1.5   # Sehr lange Wörter benötigen möglicherweise mehr Gewicht für den edit distance

    # die Gewichtung für die Häufigkeit basierend auf der Häufigkeit des Wortes im Brown corpus anpassen
    if word_freq[s1] > 10:        # Wenn das Wort sehr häufig im Korpus vorkommt, mehr Wert auf die Frequenz legen
        freq_weight = 0.2
    elif word_freq[s1] > 5:
        freq_weight = 0.15
    else:
        freq_weight = 0.1        # seltenen Wörtern eine kleinere Gewichtung für die Häufigkeit geben

    # die gewichtete Sortierung nach edit distance und -häufigkeit anwenden
    # von Kandidaten sowohl nach edit distance als auch nach Häufigkeit sortieren
    sorted_corrections = sorted(
        all_proposed_corrections,
        key=lambda w: (edit_distance(s1, w) * edit_weight) - (word_freq[w] * freq_weight)
    )

    # Top Kandidaten
    top_n = 5
    out.extend(sorted_corrections[:top_n])

    return out

###############
# Work directory festlegen
import os
current_dir = os.getcwd()
os.chdir(current_dir)
print("Mein aktuelles working directory:", os.getcwd())

# "batch0.tab" Herunterladen
import urllib.request
url = "http://aspell.net/test/cur/batch0.tab"
filename = "batch0.tab"
urllib.request.urlretrieve(url, filename)
print(f"Downloaded {filename}")
###############

Evaluation der Funktion `TEST_propose_word_corrections` – Achtung, es kann mehrere Minuten dauern, bis alle 547 Paare verarbeitet sind:

In [26]:
%%time
import csv

sumTrueCorrections = 0
sumFalseCorrections = 0
sumProposedCorrections = 0

with open('batch0.tab') as tsvfile :
  reader = csv.reader(tsvfile, delimiter='\t')
  print("{1:20}{2:20}{0:20}{3:20}".format("Gold in candidates","Test word","Gold correction","Size of candidate list"))
  print("-"*82)
  for row in reader :
        proposedCorrections = TEST_propose_word_corrections(row[0])
        sumProposedCorrections += len(proposedCorrections)
        if row[1] in proposedCorrections :
            sumTrueCorrections += 1
            print("{1:20}{2:20}{0:20}{3:20}".format("TRUE",row[0],row[1],str(len(proposedCorrections))))
        else :
            sumFalseCorrections += 1
            print("{1:20}{2:20}{0:20}{3:20}".format("FALSE",row[0],row[1],str(len(proposedCorrections))))

recallCorrections = sumTrueCorrections / (sumTrueCorrections + sumFalseCorrections)
precisionCorrections = sumTrueCorrections / sumProposedCorrections

print("Evaluation:")
print("Recall:    " + str(recallCorrections))
print("Precision: " + str(precisionCorrections))
print("F1 score:  " + str(2 * precisionCorrections * recallCorrections / (precisionCorrections + recallCorrections)))


Test word           Gold correction     Gold in candidates  Size of candidate list
----------------------------------------------------------------------------------
Accosinly           Occasionally        FALSE               5                   
Ciculer             Circler             FALSE               5                   
Circue              Circle              TRUE                5                   
Maddness            Madness             FALSE               5                   
Occusionaly         Occasionally        TRUE                5                   
Steffen             Stephen             TRUE                5                   
Thw                 The                 TRUE                5                   
Unformanlly         Unfortunately       FALSE               5                   
Unfortally          Unfortunately       FALSE               5                   
abilitey            ability             TRUE                5                   
abouy               abou

Man kann die Ergebnisse dann mit denen von Aspell vergleichen: http://aspell.net/test/cur/

# Literatur

Daniel Jurafsky & James H. Martin. 2021. Speech and Language Processing. Draft of September 21, 2021. https://web.stanford.edu/~jurafsky/slp3/.